In [3]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
from torch.optim import Adam

# Load the dataset
df = pd.read_csv("Kaggle_valid_cleansed.csv")

# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set tokenizer padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Prepare the data
class SloganDataset(Dataset):
    def __init__(self, txt_list):
        self.encodings = tokenizer(txt_list, truncation=True, padding=True, max_length=512)

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# Company name and slogan combined as input for training
train_data = [f"{row['company']} {row['description']} {row['slogan']}" for index, row in df.iterrows()]
dataset = SloganDataset(train_data)
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)


In [4]:
# Set up training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = Adam(model.parameters(), lr=5e-5)

# Training loop
model.train()
for epoch in range(3):  # Adjust epochs based on your dataset size and training need
    for batch in train_loader:
        inputs, masks = batch['input_ids'].to(device), batch['attention_mask'].to(device)
        outputs = model(inputs, attention_mask=masks, labels=inputs)
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} completed, Loss: {loss.item()}")

Epoch 1 completed, Loss: 0.31158363819122314
Epoch 2 completed, Loss: 0.35420987010002136
Epoch 3 completed, Loss: 0.3432467579841614


In [1]:
# Function to generate a slogan
def generate_slogan(company_name, description):
    model.eval()
    prompt = f"{company_name}, {description}"
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output_ids = model.generate(input_ids, max_length=20, num_return_sequences=1, no_repeat_ngram_size=2)[0]
    return tokenizer.decode(output_ids, skip_special_tokens=True)

# Example usage
company_name = "Globex Corporation"
description = "From The Simpsons, this company's mission statement is 'Evil Supervillain Front'"
slogan = generate_slogan(company_name, description)
print(f"Slogan: {slogan}")


NameError: name 'model' is not defined